# **Machine Learning Engineering - Caso de Estudio 2**

## **Ejercicio 1: Arquitectura de Datos para Actualizar la Carga de Entreno (Score de Esfuerzo)**

### **Descripción**

En este ejercicio, construiremos una arquitectura de datos que permita actualizar el **score de esfuerzo** de cada atleta cada vez que completa una actividad. Utilizaremos los datasets proporcionados para simular esta funcionalidad.

---

## **Tabla de Contenidos**

1. [Carga y Exploración de los Datasets](#Carga-y-Exploración-de-los-Datasets)
2. [Definición de la Arquitectura de Datos](#Definición-de-la-Arquitectura-de-Datos)
3. [Implementación de la Actualización del Score de Esfuerzo](#Implementación-de-la-Actualización-del-Score-de-Esfuerzo)

---

## **Carga y Exploración de los Datasets**

### **Instalación de Dependencias**

Primero, asegúrate de tener instaladas las siguientes librerías. Puedes instalarlas ejecutando la siguiente celda:

```python
!pip install pandas sqlalchemy psycopg2 scikit-learn joblib


In [11]:
# !pip install pandas sqlalchemy psycopg2 scikit-learn joblib

In [12]:
# 1. Importar Librerías Necesarias
import pandas as pd
import re
from datetime import datetime

In [13]:
# 2. Cargar los Datasets
# Asegúrate de que los archivos 'berlin_marathons_data.csv' y 'marathon_time_predictions.csv' estén en el mismo directorio que este notebook
try:
    berlin_marathons_df = pd.read_csv('Berlin_Marathon_data_1974_2019.csv')
    marathon_time_pred_df = pd.read_csv('MarathonData.csv')
    print("Datasets cargados exitosamente.")
except FileNotFoundError as e:
    print("Error: Asegúrate de que los archivos CSV estén en el directorio correcto.")
    raise e

Datasets cargados exitosamente.


C:\Users\ealda\AppData\Local\Temp\ipykernel_16604\2874771927.py:4: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  berlin_marathons_df = pd.read_csv('Berlin_Marathon_data_1974_2019.csv')


In [14]:
# 3. Exploración Inicial de los Datasets
print("\n**Berlin Marathons Data - Primeras 5 Filas:**")
display(berlin_marathons_df.head())

print("\n**Marathon Time Predictions Data - Primeras 5 Filas:**")
display(marathon_time_pred_df.head())

print("\n**Información del Dataset de Berlín Marathons:**")
berlin_marathons_df.info()

print("\n**Estadísticas Descriptivas del Dataset de Berlín Marathons:**")
display(berlin_marathons_df.describe())

print("\n**Información del Dataset de Marathon Time Predictions:**")
marathon_time_pred_df.info()

print("\n**Estadísticas Descriptivas del Dataset de Marathon Time Predictions:**")
display(marathon_time_pred_df.describe())


**Berlin Marathons Data - Primeras 5 Filas:**


,YEAR,COUNTRY,GENDER,AGE,TIME
0,1974,NaN,male,L1,02:44:53
1,1974,NaN,male,L2,02:46:43
2,1974,NaN,male,L2,02:48:08
3,1974,NaN,male,L,02:48:40
4,1974,NaN,male,L1,02:49:01



**Marathon Time Predictions Data - Primeras 5 Filas:**


,id,Marathon,Name,Category,km4week,sp4week,CrossTraining,Wall21,MarathonTime,CATEGORY
0,1,Prague17,Blair MORGAN,MAM,132.8,14.434783,NaN,1.16,2.37,A
1,2,Prague17,Robert Heczko,MAM,68.6,13.674419,NaN,1.23,2.59,A
2,3,Prague17,Michon Jerome,MAM,82.7,13.520436,NaN,1.30,2.66,A
3,4,Prague17,Daniel Or lek,M45,137.5,12.258544,NaN,1.32,2.68,A
4,5,Prague17,Luk ? Mr zek,MAM,84.6,13.945055,NaN,1.36,2.74,A



**Información del Dataset de Berlín Marathons:**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884944 entries, 0 to 884943
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   YEAR     884944 non-null  int64 
 1   COUNTRY  30796 non-null   object
 2   GENDER   884944 non-null  object
 3   AGE      872106 non-null  object
 4   TIME     884944 non-null  object
dtypes: int64(1), object(4)
memory usage: 33.8+ MB

**Estadísticas Descriptivas del Dataset de Berlín Marathons:**


,YEAR
count,884944.000000
mean,2005.176755
std,9.805638
min,1974.000000
25%,1999.000000
50%,2007.000000
75%,2013.000000
max,2019.000000



**Información del Dataset de Marathon Time Predictions:**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             87 non-null     int64  
 1   Marathon       87 non-null     object 
 2   Name           87 non-null     object 
 3   Category       81 non-null     object 
 4   km4week        87 non-null     float64
 5   sp4week        87 non-null     float64
 6   CrossTraining  13 non-null     object 
 7   Wall21         87 non-null     object 
 8   MarathonTime   87 non-null     float64
 9   CATEGORY       87 non-null     object 
dtypes: float64(3), int64(1), object(6)
memory usage: 6.9+ KB

**Estadísticas Descriptivas del Dataset de Marathon Time Predictions:**


,id,km4week,sp4week,MarathonTime
count,87.000000,87.000000,87.000000,87.000000
mean,44.000000,62.347126,139.840706,3.319080
std,25.258662,26.956019,1191.427864,0.376923
min,1.000000,17.900000,8.031414,2.370000
25%,22.500000,44.200000,11.498168,3.045000
50%,44.000000,58.800000,12.163424,3.320000
75%,65.500000,77.500000,12.854036,3.605000
max,87.000000,137.500000,11125.000000,3.980000


In [15]:
# 4. Limpieza y Preprocesamiento de Datos

## 4.1. Limpieza del Dataset de Berlín Marathons

# Función para convertir tiempo de 'HH:MM:SS' a minutos
def tiempo_a_minutos(tiempo_str):
    try:
        tiempo_obj = datetime.strptime(tiempo_str, '%H:%M:%S')
        return tiempo_obj.hour * 60 + tiempo_obj.minute + tiempo_obj.second / 60
    except:
        return None

# Aplicar la conversión a la columna 'TIME'
berlin_marathons_df['TIME_MINUTES'] = berlin_marathons_df['TIME'].apply(tiempo_a_minutos)

# Verificar la conversión
print("\n**Conversión de 'TIME' a 'TIME_MINUTES':**")
display(berlin_marathons_df[['TIME', 'TIME_MINUTES']].head())

# Manejar Valores Nulos en Berlín Marathons Data
print("\n**Valores Nulos en Berlín Marathons Data Antes de la Limpieza:**")
print(berlin_marathons_df.isnull().sum())

# Eliminar filas con NaN en columnas críticas
berlin_marathons_df = berlin_marathons_df.dropna(subset=['GENDER', 'AGE', 'TIME_MINUTES'])

print("\n**Valores Nulos en Berlín Marathons Data Después de la Limpieza:**")
print(berlin_marathons_df.isnull().sum())

# Mapeo de Categorías de Edad a Valores Numéricos
edad_mapping = {
    'L': 20,
    'L1': 25,
    'L2': 30,
    'L3': 35,
    'L4': 40,
    'L5': 45,
    'L6': 50,
    'L7': 55,
    'L8': 60
}

berlin_marathons_df['AGE_NUM'] = berlin_marathons_df['AGE'].map(edad_mapping)

print("\n**Mapeo de 'AGE' a 'AGE_NUM':**")
display(berlin_marathons_df[['AGE', 'AGE_NUM']].head())


**Conversión de 'TIME' a 'TIME_MINUTES':**


,TIME,TIME_MINUTES
0,02:44:53,164.883333
1,02:46:43,166.716667
2,02:48:08,168.133333
3,02:48:40,168.666667
4,02:49:01,169.016667



**Valores Nulos en Berlín Marathons Data Antes de la Limpieza:**
YEAR                 0
COUNTRY         854148
GENDER               0
AGE              12838
TIME                 0
TIME_MINUTES      2405
dtype: int64

**Valores Nulos en Berlín Marathons Data Después de la Limpieza:**
YEAR                 0
COUNTRY         839600
GENDER               0
AGE                  0
TIME                 0
TIME_MINUTES         0
dtype: int64

**Mapeo de 'AGE' a 'AGE_NUM':**


,AGE,AGE_NUM
0,L1,25.0
1,L2,30.0
2,L2,30.0
3,L,20.0
4,L1,25.0


In [16]:
## 4.2. Limpieza del Dataset de Marathon Time Predictions

# Verificar valores únicos en 'CrossTraining'
print("\n**Valores Únicos en 'CrossTraining':**")
print(marathon_time_pred_df['CrossTraining'].unique())

# Función para extraer horas de 'CrossTraining'
def extract_hours(cross_training):
    if pd.isnull(cross_training):
        return 0.0  # Asignar 0 si el valor es NaN
    match = re.search(r'(\d+\.?\d*)\s*h', cross_training.lower())
    if match:
        return float(match.group(1))
    else:
        return 0.0  # Asignar 0 si no se encuentra un patrón de horas

# Aplicar la función a la columna 'CrossTraining' y crear una nueva columna 'CrossTraining_Hours'
marathon_time_pred_df['CrossTraining_Hours'] = marathon_time_pred_df['CrossTraining'].apply(extract_hours)

# Verificar la extracción
print("\n**Columna 'CrossTraining_Hours' después de la extracción:**")
display(marathon_time_pred_df[['CrossTraining', 'CrossTraining_Hours']].head())

# Eliminar la columna original 'CrossTraining'
marathon_time_pred_df = marathon_time_pred_df.drop(columns=['CrossTraining'])

print("\n**Columnas Después de Eliminar 'CrossTraining':**")
print(marathon_time_pred_df.columns)

# Función para convertir valores a float en 'Wall21'
def convert_to_float(value):
    try:
        return float(value)
    except:
        return 0.0  # Asignar 0.0 si hay un error en la conversión

# Aplicar la conversión a la columna 'Wall21'
marathon_time_pred_df['Wall21'] = marathon_time_pred_df['Wall21'].apply(convert_to_float)

print("\n**Columna 'Wall21' Después de la Conversión a Float:**")
display(marathon_time_pred_df[['Wall21']].head())


**Valores Únicos en 'CrossTraining':**
[nan 'ciclista 1h' 'ciclista 4h' 'ciclista 13h' 'ciclista 5h'
 'ciclista 3h']

**Columna 'CrossTraining_Hours' después de la extracción:**


,CrossTraining,CrossTraining_Hours
0,NaN,0.0
1,NaN,0.0
2,NaN,0.0
3,NaN,0.0
4,NaN,0.0



**Columnas Después de Eliminar 'CrossTraining':**
Index(['id', 'Marathon', 'Name', 'Category', 'km4week', 'sp4week', 'Wall21',
       'MarathonTime', 'CATEGORY', 'CrossTraining_Hours'],
      dtype='object')

**Columna 'Wall21' Después de la Conversión a Float:**


,Wall21
0,1.16
1,1.23
2,1.30
3,1.32
4,1.36


In [17]:
# 5. Definición de la Arquitectura de Datos

## 5.1. Definir las Tablas Atletas y Actividades

# Definir las columnas para las tablas Atletas y Actividades
atletas_columns = ['id', 'nombre', 'edad', 'genero', 'score_esfuerzo']
atletas_df = pd.DataFrame(columns=atletas_columns)

actividades_columns = ['id', 'atleta_id', 'tipo_actividad', 'duracion_minutos', 'distancia_km', 'fecha']
actividades_df = pd.DataFrame(columns=actividades_columns)

# Mostrar las estructuras vacías
print("\n**Tabla de Atletas (Estructura Vacía):**")
display(atletas_df.head())

print("\n**Tabla de Actividades (Estructura Vacía):**")
display(actividades_df.head())


**Tabla de Atletas (Estructura Vacía):**


,id,nombre,edad,genero,score_esfuerzo



**Tabla de Actividades (Estructura Vacía):**


,id,atleta_id,tipo_actividad,duracion_minutos,distancia_km,fecha


In [18]:
## 5.2. Población Inicial de las Tablas

# Inicializar un contador para actividades
actividad_id_counter = 1

# Limitar la población a las primeras 1000 actividades para evitar problemas de rendimiento
max_actividades = 1000

# Listas para acumular nuevas filas
nuevos_atletas = []
nuevas_actividades = []

for index, row in berlin_marathons_df.iterrows():
    atleta_nombre = f"Atleta_{index + 1}"  # Generar un nombre único para el atleta
    
    # Verificar si el atleta ya está en la lista de nuevos_atletas
    if atleta_nombre not in atletas_df['nombre'].values:
        # Añadir nuevo atleta a la lista
        nuevos_atletas.append({
            'id': index + 1,
            'nombre': atleta_nombre,
            'edad': row['AGE_NUM'],
            'genero': row['GENDER'],
            'score_esfuerzo': 0  # Inicialmente 0, se actualizará
        })
    
    atleta_id = index + 1  # Asignar ID basado en el índice (ajustar según lógica real)
    
    # Registrar actividad a la lista
    nuevas_actividades.append({
        'id': actividad_id_counter,
        'atleta_id': atleta_id,
        'tipo_actividad': 'Maratón',
        'duracion_minutos': row['TIME_MINUTES'],
        'distancia_km': 42.195,
        'fecha': f"{row['YEAR']}-01-01"  # Suponemos una fecha genérica
    })
    
    # Calcular y actualizar el score de esfuerzo
    factor_distancia = 1.5
    factor_duracion = 0.5
    nuevo_score = 42.195 * factor_distancia + row['TIME_MINUTES'] * factor_duracion
    
    # Actualizar el score en la lista de nuevos_atletas
    nuevos_atletas[-1]['score_esfuerzo'] += nuevo_score
    
    # Incrementar el contador de actividades
    actividad_id_counter += 1
    
    # Limitar a las primeras 1000 actividades
    if actividad_id_counter > max_actividades:
        break

# Convertir las listas de nuevos atletas y actividades a DataFrames
nuevos_atletas_df = pd.DataFrame(nuevos_atletas)
nuevas_actividades_df = pd.DataFrame(nuevas_actividades)

# Concatenar los nuevos atletas y actividades a los DataFrames principales
atletas_df = pd.concat([atletas_df, nuevos_atletas_df], ignore_index=True)
actividades_df = pd.concat([actividades_df, nuevas_actividades_df], ignore_index=True)

print("\n**Tabla de Atletas Después de la Población Inicial (Primeras 1000 Actividades):**")
display(atletas_df.head())

print("\n**Tabla de Actividades Después de la Población Inicial (Primeras 1000 Actividades):**")
display(actividades_df.head())


**Tabla de Atletas Después de la Población Inicial (Primeras 1000 Actividades):**


C:\Users\ealda\AppData\Local\Temp\ipykernel_16604\2330937067.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  atletas_df = pd.concat([atletas_df, nuevos_atletas_df], ignore_index=True)
C:\Users\ealda\AppData\Local\Temp\ipykernel_16604\2330937067.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  actividades_df = pd.concat([actividades_df, nuevas_actividades_df], ignore_index=True)


,id,nombre,edad,genero,score_esfuerzo
0,1,Atleta_1,25.0,male,145.734167
1,2,Atleta_2,30.0,male,146.650833
2,3,Atleta_3,30.0,male,147.359167
3,4,Atleta_4,20.0,male,147.625833
4,5,Atleta_5,25.0,male,147.800833



**Tabla de Actividades Después de la Población Inicial (Primeras 1000 Actividades):**


,id,atleta_id,tipo_actividad,duracion_minutos,distancia_km,fecha
0,1,1,Maratón,164.883333,42.195,1974-01-01
1,2,2,Maratón,166.716667,42.195,1974-01-01
2,3,3,Maratón,168.133333,42.195,1974-01-01
3,4,4,Maratón,168.666667,42.195,1974-01-01
4,5,5,Maratón,169.016667,42.195,1974-01-01


In [19]:
# 6. Implementación de la Actualización del Score de Esfuerzo

## 6.1. Definir la Función para Registrar Nuevas Actividades y Actualizar el Score de Esfuerzo

def registrar_actividad(atletas_df, actividades_df, atleta_nombre, tipo_actividad, duracion_minutos, distancia_km, fecha=None):
    """
    Registra una nueva actividad para un atleta y actualiza su score de esfuerzo.
    
    Parámetros:
    - atletas_df: DataFrame de atletas.
    - actividades_df: DataFrame de actividades.
    - atleta_nombre: Nombre del atleta (string).
    - tipo_actividad: Tipo de actividad (string).
    - duracion_minutos: Duración de la actividad en minutos (float).
    - distancia_km: Distancia de la actividad en kilómetros (float).
    - fecha: Fecha de la actividad (string en formato 'YYYY-MM-DD'). Si es None, se usa la fecha actual.
    
    Retorna:
    - atletas_df: DataFrame de atletas actualizado.
    - actividades_df: DataFrame de actividades actualizado.
    """
    global actividad_id_counter
    
    # Verificar si el atleta existe
    if atletas_df[atletas_df['nombre'] == atleta_nombre].empty:
        print(f"Atleta '{atleta_nombre}' no encontrado. Creando un nuevo atleta.")
        # Asignar un nuevo ID
        nuevo_id = atletas_df['id'].max() + 1 if not atletas_df.empty else 1
        # Añadir nuevo atleta con edad y género por defecto o puedes modificar para recibir estos parámetros
        nuevo_atleta = pd.DataFrame([{
            'id': nuevo_id,
            'nombre': atleta_nombre,
            'edad': 25,  # Valor por defecto
            'genero': 'male',  # Valor por defecto
            'score_esfuerzo': 0
        }])
        atletas_df = pd.concat([atletas_df, nuevo_atleta], ignore_index=True)
    
    # Obtener el ID del atleta
    atleta_id = atletas_df[atletas_df['nombre'] == atleta_nombre]['id'].values[0]
    
    # Asignar fecha actual si no se proporciona
    if fecha is None:
        fecha = datetime.now().strftime("%Y-%m-%d")
    
    # Crear un nuevo registro de actividad
    nueva_actividad = pd.DataFrame([{
        'id': actividad_id_counter,
        'atleta_id': atleta_id,
        'tipo_actividad': tipo_actividad,
        'duracion_minutos': duracion_minutos,
        'distancia_km': distancia_km,
        'fecha': fecha
    }])
    
    # Añadir la nueva actividad al DataFrame de actividades
    actividades_df = pd.concat([actividades_df, nueva_actividad], ignore_index=True)
    
    # Calcular nuevo score de esfuerzo
    factor_distancia = 1.5
    factor_duracion = 0.5
    nuevo_score = distancia_km * factor_distancia + duracion_minutos * factor_duracion
    
    # Actualizar el score de esfuerzo del atleta
    atletas_df.loc[atletas_df['id'] == atleta_id, 'score_esfuerzo'] += nuevo_score
    
    print(f"Actividad registrada para '{atleta_nombre}'. Nuevo score de esfuerzo: {atletas_df.loc[atletas_df['id'] == atleta_id, 'score_esfuerzo'].values[0]:.2f}")
    
    # Incrementar el contador de actividades
    actividad_id_counter += 1
    
    return atletas_df, actividades_df

In [20]:
## 6.2. Ejemplos de Uso de la Función

# Registrar una nueva actividad para un atleta existente
atletas_df, actividades_df = registrar_actividad(
    atletas_df, 
    actividades_df, 
    atleta_nombre='Atleta_1', 
    tipo_actividad='Correr al aire libre', 
    duracion_minutos=60, 
    distancia_km=10, 
    fecha='2024-10-20'
)

# Registrar una nueva actividad para un atleta nuevo
atletas_df, actividades_df = registrar_actividad(
    atletas_df, 
    actividades_df, 
    atleta_nombre='Atleta_1001', 
    tipo_actividad='Caminadora', 
    duracion_minutos=45, 
    distancia_km=8, 
    fecha='2024-10-21'
)

Actividad registrada para 'Atleta_1'. Nuevo score de esfuerzo: 190.73
Actividad registrada para 'Atleta_1001'. Nuevo score de esfuerzo: 228.05


In [21]:
# Verificar las actualizaciones
print("\n**Últimos Registros en la Tabla de Atletas:**")
display(atletas_df.tail())

print("\n**Últimos Registros en la Tabla de Actividades:**")
display(actividades_df.tail())


**Últimos Registros en la Tabla de Atletas:**


,id,nombre,edad,genero,score_esfuerzo
995,1038,Atleta_1038,NaN,male,148.217500
996,1039,Atleta_1039,NaN,male,148.275833
997,1040,Atleta_1040,NaN,male,148.375833
998,1041,Atleta_1041,NaN,male,149.492500
999,1042,Atleta_1042,NaN,male,149.767500



**Últimos Registros en la Tabla de Actividades:**


,id,atleta_id,tipo_actividad,duracion_minutos,distancia_km,fecha
997,998,1040,Maratón,170.166667,42.195,1979-01-01
998,999,1041,Maratón,172.400000,42.195,1979-01-01
999,1000,1042,Maratón,172.950000,42.195,1979-01-01
1000,1001,1,Correr al aire libre,60.000000,10.000,2024-10-20
1001,1002,1001,Caminadora,45.000000,8.000,2024-10-21




### 7. Conclusión y Próximos Pasos

Hemos definido una arquitectura de datos utilizando pandas DataFrames para simular tablas de atletas y actividades. Además, implementamos una función para registrar nuevas actividades y actualizar el score de esfuerzo de los atletas.

### Próximos Pasos:
1. **Persistencia de Datos:**
   - Utilizar una base de datos relacional como PostgreSQL para almacenar las tablas de atletas y actividades de manera persistente.
   - Conectar a la base de datos usando SQLAlchemy y migrar las tablas desde pandas a la base de datos.

2. **Automatización:**
   - Implementar scripts que se ejecuten automáticamente cada vez que se registre una nueva actividad, actualizando el score de esfuerzo en la base de datos.
   - Utilizar herramientas como GitHub Actions o cron jobs para programar tareas automáticas.

3. **Validación y Manejo de Errores:**
   - Añadir validaciones adicionales en la función `registrar_actividad` para manejar casos especiales, como actividades con distancias o duraciones no realistas.
   - Implementar manejo de excepciones para garantizar que el sistema sea robusto frente a entradas inválidas.

4. **Integración con Otros Sistemas:**
   - Integrar la arquitectura de datos con sistemas de front-end o APIs que permitan a los usuarios registrar sus actividades de manera interactiva.

5. **Documentación:**
   - Documentar cada parte del proceso para facilitar futuras mejoras y colaboraciones.

6. **Escalabilidad:**
   - Considerar cómo escalar la solución para manejar un mayor volumen de datos y usuarios, optimizando consultas y almacenamiento en la base de datos.
